In [ ]:
%cd ./
%rm -rf darknet

In [ ]:
#we clone the fork of darknet maintained by roboflow
#small changes have been made to configure darknet for training
!curl -L "https://drive.google.com/uc?id=1Oc6pQg5jSQUzmHkbw2LQzld5tozjUXmx" > darknet.zip
!unzip -q darknet.zip
!rm darknet.zip

In [ ]:
!mv darknet-master darknet

In [ ]:
#change directory to darknet
%cd darknet/

#change makefile to have a gpu and opencv
%cd darknet
!sed -i 

In [ ]:
# make darknet
! make

In [ ]:
#download the newly released yolov4-tiny weights
%cd ./darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
#if you already have YOLO darknet format, you can skip this step
#otherwise we recommend formatting in Roboflow
%cd ./darknet
!curl -L "https://app.roboflow.com/ds/MwqoUvnApc?key=LRDwgzBXrw" > roboflow.zip; unzip -q roboflow.zip; rm roboflow.zip

In [ ]:
#Set up training file directories for custom dataset
%cd ./darknet/
%cp train/_darknet.labels data/obj.names
%mkdir data/obj
#copy image and labels
%cp train/*.jpg data/obj/
%cp valid/*.jpg data/obj/

%cp train/*.txt data/obj/
%cp valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
    out.write('classes = 2\n')
    out.write('train = data/train.txt\n')
    out.write('valid = data/valid.txt\n')
    out.write('names = data/obj.names\n')
    out.write('backup = backup/')

#write train file (just the image list)
import os

with open('data/train.txt', 'w') as out:
    for img in [f for f in os.listdir('train') if f.endswith('jpg')]:
        out.write('data/obj/' + img + '\n')

#write the valid file (just the image list)
import os

with open('data/valid.txt', 'w') as out:
    for img in [f for f in os.listdir('valid') if f.endswith('jpg')]:
        out.write('data/obj/' + img + '\n')

In [ ]:
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

num_classes = file_len('train/_darknet.labels')
max_batches = num_classes*2000
steps1 = .8 * max_batches
steps2 = .9 * max_batches
steps_str = str(steps1)+','+str(steps2)
num_filters = (num_classes + 5) * 3


print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))

#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists('./cfg/custom-yolov4-tiny-detector.cfg'): os.remove('./cfg/custom-yolov4-tiny-detector.cfg')


#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
#coco.names is hardcoded somewhere in the detector
%cp data/obj.names data/coco.names

In [ ]:
# collect pretrained weights
!curl -L "https://drive.google.com/uc?id=1b3Z0sbJTOr0YFtDu_6z3EC0rGtBpVEmS" > weights.zip
!unzip -q weights.zip
!rm weights.zip

In [ ]:
#define utility function
def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.imshow(cv2.imread(path))
    plt.show()

In [ ]:
!echo > result.txt

In [ ]:
!mkdir result_img

In [ ]:
#/test has images that we can test our detector on
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random
img_path = "test/" + random.choice(test_images);

#test out our detector!
!./darknet detect custom-yolov4-tiny-detector.cfg custom-yolov4-tiny-detector_best.weights -ext_output {img_path} result.txt

In [ ]:
!ls result_img

In [ ]:
imShow('predictions.jpg')